<a href="https://colab.research.google.com/github/glnrzr/TezCalismasi/blob/master/calisma1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
cd /content/drive/Colab Notebooks/full_version

In [0]:
import numpy as np
import os

miniData = []
filenames = ['ekonomi100k.txt', 'politika100k.txt', 'spor100k.txt']
for fname in filenames:
    with open(fname) as infile:
        for line in infile:
            try:
              _Y, _X = line.strip().split("\t")
              miniData.append([_X, _Y])
            except Exception as e:
              print line
              #print(e)
              pass
np.random.shuffle(miniData)
train_x = [x[0] for x in miniData]
train_y = np.asarray([x[1] for x in miniData])
miniData = None
#print train_x[0] , " " , train_y[0]
#print train_x[1] , " " , train_y[1]

num_of_classes = len(np.unique(train_y))

In [0]:
i = 0
for val in train_y:
  if val == '3':
    i = i+1
print i


In [0]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(train_y)
encoded_Y = encoder.transform(train_y)
# convert integers to dummy variables (i.e. one hot encoded)
train_y = np_utils.to_categorical(encoded_Y)

In [0]:
import json
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer

max_words = 3000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_x)
dictionary = tokenizer.word_index

with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)


def convert_text_to_index_array(text):
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]


allWordIndices = []
for text in train_x:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

allWordIndices = np.asarray(allWordIndices)
train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')
#train_y = keras.utils.to_categorical(train_y, 4)

In [0]:
convert_text_to_index_array("komutan suriyede bin askerimiz var dogal")

In [0]:
import h5py
with h5py.File('train_x.h5', 'w') as hf:
    hf.create_dataset("three-class-all",  data=train_x)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(num_of_classes, activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'])

model.fit(train_x, train_y,
  batch_size=32,
  epochs=5,
  verbose=1,
  validation_split=0.1,
  shuffle=True)

In [0]:
#save the model
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')

In [0]:
def turkcelestir(metin):
  metin = metin.decode("utf-8") # because u'string'.translate
  turkce = {ord(c): ord(t) for c, t in zip(u"ÇĞİÖŞÜçğıöşü", u"CGIOSUcgiosu")}
  #print metin.translate(turkce)
  return metin.translate(turkce)

In [0]:
turkcelestir('ömer çaydanlık Ğözde')
txt1 = 'Bingöl de 24 Mayıs 1993 tarihinde'
print turkcelestir(txt1)
#txt1 = txt1.replace('\n', ' ')

In [0]:
import json
import numpy as np
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json

# we're still going to use a Tokenizer here, but we don't need to fit it
tokenizer = Tokenizer(num_words=3000)
# for human-friendly printing
labels = ['politika', 'spor', 'ekonomi']

# read in our saved dictionary
with open('dictionary.json', 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

# this utility makes sure that all the words in your input
# are registered in the dictionary
# before trying to turn them into a matrix.
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
        else:
            print("'%s' not in training corpus; ignoring." %(word))
    return wordIndices

# read in your saved model structure
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
# and create a model from that
model = model_from_json(loaded_model_json)
# and weight your nodes with your saved values
model.load_weights('model.h5')

# okay here's the interactive part
while 1:
    evalSentence = raw_input('Test için bir cümle girin, çıkmak için Enter: ')

    if len(evalSentence) == 0:
        break

    # format your input for the neural net
    evalSentence = turkcelestir(evalSentence)
    testArr = convert_text_to_index_array(evalSentence)
    input = tokenizer.sequences_to_matrix([testArr], mode='binary')
    # predict which bucket your input belongs in
    pred = model.predict(input)
    # and print it for the humons
    print("Tahmin edilen sınıf: %s ; %f%% confidence" % (labels[np.argmax(pred)], pred[0][np.argmax(pred)] * 100))